In [ ]:
import pandas as pd
import sqlite3
import bz2

# url = "http://awing.kis.agh.edu.pl:8080/detectors_names_traffic_s_small.csv.bz2"
# file_name = "detectors.csv.bz2"

# import urllib.request
# urllib.request.urlretrieve(url, file_name)

# with bz2.open(file_name, "rt") as f:
#     df = pd.read_csv(f)

In [ ]:
conn = sqlite3.connect("proj6_readings.sqlite")
# df.to_sql("readings", conn, if_exists="replace", index=False)

In [ ]:
# cur = conn.cursor()

# cur.execute("""CREATE INDEX detector_id ON readings (detector_id);""")
# cur.execute("""CREATE INDEX starttime ON readings (starttime);""")
# conn.commit()

In [ ]:
# import pandas as pd
# import sqlite3
# import bz2

# conn = sqlite3.connect("proj6_readings.sqlite")


In [9]:
ex1 = pd.read_sql("SELECT count(DISTINCT detector_id) AS 'count(*)' FROM readings", conn)
print(ex1.shape)
ex1.to_pickle("proj6_ex01_detector_no.pkl")
print(ex1)

(1, 1)
   count(*)
0       146


In [10]:
ex2 = pd.read_sql("SELECT detector_id, count(detector_id) AS 'count(count)', min(starttime), max(starttime) FROM readings GROUP BY detector_id", conn)
ex2.to_pickle("proj6_ex02_detector_stat.pkl")
print(ex2)

     detector_id  count(count)       min(starttime)       max(starttime)
0              1        357214  2015-06-22 23:58:31  2016-07-12 23:57:01
1              2        357201  2015-06-22 23:58:31  2016-07-12 23:57:01
2              3        356755  2015-06-22 23:58:31  2016-07-12 23:57:01
3              4        357213  2015-06-22 23:58:31  2016-07-12 23:57:01
4              5        357204  2015-06-22 23:58:31  2016-07-12 23:57:01
..           ...           ...                  ...                  ...
141          142        352730  2015-06-22 23:58:31  2016-07-12 23:57:01
142          143        352729  2015-06-22 23:58:31  2016-07-12 23:57:01
143          144        352730  2015-06-22 23:58:31  2016-07-12 23:57:01
144          145        352730  2015-06-22 23:58:31  2016-07-12 23:57:01
145          146        147666  2015-06-22 23:58:31  2015-12-01 04:21:01

[146 rows x 4 columns]


In [11]:
ex3 = pd.read_sql("""
                    SELECT 
                        detector_id,
                        count,
                        LAG(count) OVER (PARTITION BY detector_id ORDER BY starttime) AS prev_count
                    FROM readings
                    WHERE detector_id = 146
                    ORDER BY starttime
                    LIMIT 500;
                  """, conn)
ex3.to_pickle("proj6_ex03_detector_146_lag.pkl")
print(ex3)

     detector_id  count  prev_count
0            146      1         NaN
1            146      0         1.0
2            146      0         0.0
3            146      0         0.0
4            146      0         0.0
..           ...    ...         ...
495          146      0         0.0
496          146      2         0.0
497          146      2         2.0
498          146      2         2.0
499          146      2         2.0

[500 rows x 3 columns]


In [ ]:
ex4 = pd.read_sql("""
                    SELECT 
                        detector_id,
                        count,
                        SUM(count) OVER (
                            PARTITION BY detector_id 
                            ORDER BY starttime 
                            ROWS BETWEEN CURRENT ROW AND 10 FOLLOWING
                        ) AS window_sum
                    FROM readings
                    WHERE detector_id = 146
                    ORDER BY starttime
                    LIMIT 500;
                    """, conn)
ex4.to_pickle("proj6_ex04_detector_146_sum.pkl")
print(ex4)

     detector_id  count  window_sum            starttime              endtime
0            146      1           2  2015-06-22 23:58:31  2015-06-23 00:00:01
1            146      0           2  2015-06-23 00:00:01  2015-06-23 00:01:31
2            146      0           2  2015-06-23 00:01:31  2015-06-23 00:03:01
3            146      0           2  2015-06-23 00:03:01  2015-06-23 00:04:31
4            146      0           2  2015-06-23 00:04:31  2015-06-23 00:06:01
..           ...    ...         ...                  ...                  ...
495          146      0          20  2015-06-23 12:21:01  2015-06-23 12:22:31
496          146      2          22  2015-06-23 12:22:31  2015-06-23 12:24:01
497          146      2          21  2015-06-23 12:24:01  2015-06-23 12:25:31
498          146      2          21  2015-06-23 12:25:31  2015-06-23 12:27:01
499          146      2          21  2015-06-23 12:27:01  2015-06-23 12:28:31

[500 rows x 5 columns]


In [ ]:
conn.close()